<a href="https://colab.research.google.com/github/ahoucbvtw/NewsClassification-Chinese/blob/main/NewsClassification-Chinese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
path = "drive/My Drive/News/chinese_news_trans.zip"
f = zipfile.ZipFile(path)
f.extractall("train")
path = "drive/My Drive/News/chinese_news_test.zip"
f = zipfile.ZipFile(path)
f.extractall("test")

In [2]:
# "train/chinese_news_trans/*"
import os
import pandas as pd
import glob
contents = []
types = []
def showdata(path):
    for dirName in glob.glob(os.path.join(path, "*")):
        t = os.path.split(dirName)[-1]
        for fn in glob.glob(os.path.join(dirName, "*.txt")) + glob.glob(os.path.join(dirName, "*.TXT")):
            with open(fn) as f:
                contents.append(f.read())
                types.append(t)
    data = pd.DataFrame({"content": contents, "answer": types})
    return data

In [3]:
train_data = showdata("train/chinese_news_trans")
test_data = showdata("test/chinese_news_test")

train_data
# test_data
# print(test_df)

,content,answer
0,日月光華 -- SE精華區文章閱讀\n---------------------------...,計算機
1,日月光華 -- Software精華區文章閱讀\n---------------------...,計算機
2,日月光華 -- Hardware精華區文章閱讀\n---------------------...,計算機
3,日月光華 -- Hardware精華區文章閱讀\n---------------------...,計算機
4,日月光華 -- SE精華區文章閱讀\n---------------------------...,計算機
...,...,...
2632,膠濟鐵路複線二期工程正加緊施工\n新華社濟南５月１４日電（通訊員王建國...,交通
2633,【 日 期 】19960612\n【 版 號 】9\n【 標 題 】認真實施分道航行規...,交通
2634,"\n ★★ 新華社電訊稿數據庫 檢索命中 44375 篇, 顯示第 3104 篇。...",交通
2635,濟南鐵路局積極做好防洪準備工作\n新華社濟南５月２６日電（通訊員孔鑑、...,交通


In [4]:
# 將dataset中的答案統一區分並列出
newsType = train_data["answer"].unique()
newsType_dict = { n:i for i,n in enumerate(newsType)}
newsType_r_dict = { i:n for i,n in enumerate(newsType_dict)}
newsType_dict

{'交通': 9,
 '政治': 8,
 '教育': 7,
 '環境': 1,
 '經濟': 6,
 '藝術': 3,
 '計算機': 0,
 '軍事': 4,
 '醫藥': 2,
 '體育': 5}

In [5]:
#將answer轉換成0123...
train_data["answer"] = train_data["answer"].replace(newsType_dict)
test_data["answer"] = test_data["answer"].replace(newsType_dict)

train_data

,content,answer
0,日月光華 -- SE精華區文章閱讀\n---------------------------...,0
1,日月光華 -- Software精華區文章閱讀\n---------------------...,0
2,日月光華 -- Hardware精華區文章閱讀\n---------------------...,0
3,日月光華 -- Hardware精華區文章閱讀\n---------------------...,0
4,日月光華 -- SE精華區文章閱讀\n---------------------------...,0
...,...,...
2632,膠濟鐵路複線二期工程正加緊施工\n新華社濟南５月１４日電（通訊員王建國...,9
2633,【 日 期 】19960612\n【 版 號 】9\n【 標 題 】認真實施分道航行規...,9
2634,"\n ★★ 新華社電訊稿數據庫 檢索命中 44375 篇, 顯示第 3104 篇。...",9
2635,濟南鐵路局積極做好防洪準備工作\n新華社濟南５月２６日電（通訊員孔鑑、...,9


In [6]:
# 新創一Function將新聞內部的換行符號刪除並與之前資料表結合
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
def newscut(news):
    news = news.replace("\r", "").replace("\n", "").replace("\n\n","")
    return " ".join(jieba.cut(news))

train_data["content"] = train_data["content"].apply(newscut)
test_data["content"] = test_data["content"].apply(newscut)
train_data

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.739 seconds.
Prefix dict has been built successfully.


,content,answer
0,日月 光華 -- SE 精華區 文章 閱讀 --------------------...,0
1,日月 光華 -- Software 精華區 文章 閱讀 --------------...,0
2,日月 光華 -- Hardware 精華區 文章 閱讀 --------------...,0
3,日月 光華 -- Hardware 精華區 文章 閱讀 --------------...,0
4,日月 光華 -- SE 精華區 文章 閱讀 --------------------...,0
...,...,...
2632,膠濟 鐵路 複線 二期工程 正 加緊 施工 新華...,9
2633,【 日 期 】 19960612 【 版 號 】 9 【 ...,9
2634,★ ★ 新華社 電訊稿 數據庫 檢索 命中 44375 ...,9
2635,濟南鐵路局 積極 做好 防洪 準備 工作 新華社...,9


In [7]:
# countvectorizer
# 訓練資料=> fit(統計有多少分詞標籤)/transform(統計這些分詞標籤出現次數),所以訓練資料2個都要 = fit_transform
# 測試資料=> transform(測試資料不須再做分詞標籤，因為只須透過訓練分詞來分辨判斷，新的分詞就不理)
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
train_count =  vec.fit_transform(train_data["content"])
test_count = vec.transform(test_data["content"])
#訓練資料裡每首詩對每個分詞欄位出現的次數
print(train_count)
# vec.vocabulary_ => 字典
# 利用vec.vocabulary_[","]等等標點符號或換行是否還在裡面
#vec.vocabulary_ (每個train_count中分詞欄位對應的分詞文字)

#測試資料裡每首詩對每個分詞欄位出現的次數
print(test_count)

  (0, 39035)	4
  (0, 14348)	4
  (0, 5567)	3
  (0, 54943)	1
  (0, 38259)	1
  (0, 69887)	1
  (0, 50877)	1
  (0, 4177)	4
  (0, 50271)	2
  (0, 65866)	2
  (0, 62649)	7
  (0, 50878)	1
  (0, 818)	1
  (0, 25)	2
  (0, 59)	1
  (0, 202)	1
  (0, 1467)	1
  (0, 0)	1
  (0, 39335)	1
  (0, 12931)	1
  (0, 11491)	1
  (0, 38181)	1
  (0, 65867)	1
  (0, 62492)	2
  (0, 19422)	1
  :	:
  (2636, 15401)	2
  (2636, 46096)	4
  (2636, 11197)	1
  (2636, 61805)	1
  (2636, 46094)	2
  (2636, 20701)	1
  (2636, 47953)	3
  (2636, 25337)	4
  (2636, 66766)	3
  (2636, 52729)	2
  (2636, 65353)	1
  (2636, 70612)	1
  (2636, 56569)	1
  (2636, 42880)	1
  (2636, 66857)	1
  (2636, 38335)	1
  (2636, 74149)	3
  (2636, 28041)	1
  (2636, 71035)	1
  (2636, 65077)	1
  (2636, 18309)	1
  (2636, 63528)	1
  (2636, 42044)	1
  (2636, 31832)	1
  (2636, 58179)	1
  (0, 0)	1
  (0, 19)	1
  (0, 25)	2
  (0, 59)	1
  (0, 129)	1
  (0, 202)	1
  (0, 443)	1
  (0, 801)	1
  (0, 818)	1
  (0, 842)	1
  (0, 906)	1
  (0, 1400)	1
  (0, 1467)	1
  (0, 1527)	1
  (0, 

In [8]:
# 3種單純貝氏'
# Gaussian: 連續特徵=>高斯分佈(常態分佈:可以少少的資料就做出一條全部符合此常態方程式,ex:x=4.1、4.11、4.111...都可以透過此常態方程式算出各個機率)
# Muitinomial: 整數特徵(文字)
# Beronull: 只有「是否」2種特徵
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train_count, train_data["answer"])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
#利用測試資料驗證計算出來的模型公式好壞
from sklearn.metrics import accuracy_score
pre = clf.predict(test_count)
print("預測結果:", list(pre))
print("正確結果:", list(test_data["answer"]))
print("正確率:", accuracy_score(pre, test_data["answer"]))

預測結果: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 8, 1, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 1, 1, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [21]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(test_data["answer"], pre), columns = [n + ("predict") for n in newsType], index = [n + ("true") for n in newsType])

,計算機predict,環境predict,醫藥predict,藝術predict,軍事predict,體育predict,經濟predict,教育predict,政治predict,交通predict
計算機true,182,0,0,0,0,0,4,0,0,0
環境true,0,187,0,0,0,0,1,0,4,0
醫藥true,0,0,187,1,0,0,6,3,7,0
藝術true,0,0,0,240,0,0,0,0,2,0
軍事true,0,0,0,1,199,0,0,3,37,0
體育true,0,0,0,0,0,448,0,0,1,0
經濟true,0,0,0,1,0,0,306,0,3,0
教育true,0,0,0,2,0,1,0,212,1,0
政治true,0,0,0,0,1,0,0,0,504,0
交通true,0,0,0,0,1,0,2,0,1,190


In [17]:
#利用使用者輸入新聞內容來預測是屬於哪類文章? 環境、經濟...
s = input("輸入新聞內容:")
#當要predict時一定要先轉群集(list)
inputs = [newscut(s)]
x = vec.transform(inputs)
#依照使用者輸入的預測為屬於哪類文章
probality = clf.predict_proba(x)[0]
print("所有類型文章的預測機率:", probality)
print("使用者輸入:", inputs)

for name, prob in zip(newsType, probality):
    print(name, "的預測機率:", round(prob, 2))
userpre = clf.predict(x)[0]
# argmax: 找最大值的index
print(f"預測結果此文章類型為『{newsType_r_dict[probality.argmax()]}』最高！！")

輸入新聞內容:國內新增本土個案13例，新北獅子會群聚新增9例個案，四例與台北市萬華區茶藝館群聚相關。指揮中心指揮官陳時中表示，今天新增的案例基本上都和萬華進出有關係。  陳時中表示，案1245、案1246、案1248、案1250、案1251、案1253及案1255至案1257等9人，為7男2女，年齡介於30多歲至80多歲，其中4人無症狀，5人曾有症狀，發病日介於5月6日至5月11日。9人因曾與案1203於同餐廳用餐，匡列為接觸者並進行採檢，於今日確診；截至目前案1203相關群聚共計20人確診。  案1247、案1252及案1254為40多歲女性、30多歲女性及50多歲女性，分別於臺北市萬華區3家不同之茶藝館工作，3人於5月8日至5月10日間陸續出現症狀，並曾至診所就醫，後續分別於5月11日、5月12日至醫院採檢，於今日確診。  另外，案1249為60多歲男性，5月5日出現乾咳、打噴嚏情形，5月7日出現發燒、肌肉痠痛、喉嚨痛及頭痛症狀，5月10日至診所就醫；個案因持續發燒，5月12日至醫院就醫採檢，於今日確診。衛生單位初步調查，個案會定期於臺北市萬華區活動，研判個案與萬華區茶藝館群聚有地緣相關性；已初步掌握個案接觸者3人，列居家隔離並安排採檢。截至目前萬華茶藝館相關群聚共計7人確診。  陳時中進一步表示，案1203群聚相關個案中，有部分個案曾至臺北市萬華區茶藝館活動；另萬華區茶藝館群聚相關個案中，有個案曾至宜蘭縣礁溪鄉信義路「巨城電玩」遊藝場及宜蘭市新民路「大地球」遊藝場活動，因該2家遊藝場為羅東「銀河百家樂」遊藝場之分店，衛生單位將持續深入調查，以釐清該三起群聚事件的關聯性。  陳時中提醒，個案於可傳染期間曾有公共場所活動史，請曾於附件所列時間及場所活動的民眾，進行14天自我健康監測，如有疑似症狀，儘速戴口罩至指定社區採檢院所就醫，不得搭乘大眾運輸，就醫時請主動告知活動史、接觸史等資訊。  截至目前國內有1256例確診，分別為1068例境外移入，135例本土病例，36例敦睦艦隊、2例航空器感染、1例不明及14例調查中；另案530移除為空號。確診個案中12人死亡、、142人住院隔離中。
所有類型文章的預測機率: [3.41103953e-30 3.76793699e-15 1.00000000e+00 6.54538881e-34
 1.18613628e-27 2.